Steps to cover
1. Read the dataset and drop col "Patient ID"
2. As the dataset contains huge amount of null values; select only those columns which contains <= OR < 80% null values
3. Combine the columns 'Influenza A' and 'Influenza B' by using the below condition. 

if both detected: 3
if InfA detected: 2
if InfB detected: 1
if both not detected: 0
else: -1

Create a new column name as 'Influenza(AB)' and add the new values in it. Drop the previous two cols

4.  Now use the column 'ParaInfluenza 1','ParaInfluenza 2', 'ParaInfluenza 3' 'ParaInfluenza 4'. and create the conditions as 

('detected', 'not_detected', 'not_detected', 'not_detected'), = 0
    ('not_detected', 'detected', 'not_detected', 'not_detected'), = 1
    ('not_detected', 'not_detected', 'detected', 'not_detected'), = 2
    ('not_detected', 'not_detected', 'not_detected', 'detected') = 3

Add the new values in a new column as 'ParaInfluenza(1,2,3,4)' and drop the previous ones..

4. Repeat same step for cols 'CoronavirusNL63' , 'Coronavirus HKU1', 'Coronavirus229E' , 'CoronavirusOC43'

5. Replace all null values with a unique value such as -1

6. and save it in a new dataframe file either csv or xlsx

In [1]:
# import covid-19 excel spreadsheet

import pandas as pd

df = pd.read_excel('dataset.xlsx')

df = df.drop(columns='Patient ID')

In [2]:
# combine influenza virus infection columns
df['InfluenzaAB'] = df.apply(lambda r: -1 if r['Influenza A']=='' and r['Influenza B']==''
                           else 1 if r['Influenza A']=='detected' and r['Influenza B']=='not_detected'
                           else 2 if r['Influenza A']=='not_detected' and r['Influenza B']=='detected'
                           else 3 if r['Influenza A']=='detected' and r['Influenza B']=='not_detected'
                           else 0, axis=1)
df.drop(columns='Influenza A',inplace=True)
df.drop(columns='Influenza B',inplace=True)

### Combine Parainfluenza 1,2,3,4 Columns

In [3]:
def ParainfluenzaId(one, two, three, four):
    if one=='detected':
        return 1
    elif two=='detected':
        return 2
    elif three=='detected':
        return 3
    elif four=='detected':
        return 4
    elif one=='not_detected' and two=='not_detected' and three=='not_detected' and four=='not_detected':
        return 0
    else:
        return -1

In [4]:
## Function for Corona virus reductions
def CoronaId(c_nl63, c_hku1, c_22e2, c_oc43):
    if c_nl63=='detected':
        return 1
    elif c_hku1=='detected':
        return 2
    elif c_22e2=='detected':
        return 3
    elif c_oc43=='detected':
        return 4
    elif c_nl63=='not_detected' and c_hku1=='not_detected' and c_22e2=='not_detected' and c_oc43=='not_detected':
        return 0
    else:
        return -1


In [5]:
# loop data frame and combine Parainfluenza columns into one column and drop all Parainfluenza columns


jj = len( df['Parainfluenza 1'])

pp  = []
for x in range(jj):
    jj = ParainfluenzaId(df['Parainfluenza 1'][x],
                                                 df['Parainfluenza 2'][x],
                                                 df['Parainfluenza 3'][x],
                                                 df['Parainfluenza 4'][x])
    pp.append(jj)

pp2 = pd.DataFrame(pp)
# df['Parainfluenza1234'] = para1234
# df.add(para1234)
df['Parainfluenza1234'] = pp2
df['Parainfluenza1234']
df.drop(columns='Parainfluenza 1',inplace=True)
df.drop(columns='Parainfluenza 2',inplace=True)
df.drop(columns='Parainfluenza 3',inplace=True)
df.drop(columns='Parainfluenza 4',inplace=True)


### Coronavirus Columns Reductions

In [6]:
jj = len( df['Coronavirus229E'])
jj
cpp  = []
for x in range(jj):
    jj = CoronaId(df['CoronavirusNL63'][x],
                                                 df['Coronavirus HKU1'][x],
                                                 df['Coronavirus229E'][x],
                                                 df['CoronavirusOC43'][x])
    cpp.append(jj)

cpp2 = pd.DataFrame(cpp)
# df['Parainfluenza1234'] = para1234
# df.add(para1234)
df['Corona'] = cpp2
df['Corona']
df.drop(columns='CoronavirusNL63',inplace=True)
df.drop(columns='Coronavirus HKU1',inplace=True)
df.drop(columns='Coronavirus229E',inplace=True)
df.drop(columns='CoronavirusOC43',inplace=True)


In [7]:
# Get null value count
null_counts = df.isna().sum()
null_counts

Patient age quantile                                        0
SARS-Cov-2 exam result                                      0
Patient addmited to regular ward (1=yes, 0=no)              0
Patient addmited to semi-intensive unit (1=yes, 0=no)       0
Patient addmited to intensive care unit (1=yes, 0=no)       0
                                                         ... 
Phosphor                                                 5624
ctO2 (arterial blood gas analysis)                       5617
InfluenzaAB                                                 0
Parainfluenza1234                                           0
Corona                                                      0
Length: 103, dtype: int64

In [8]:
# calcaulate percentages of null columns
null_percents = null_counts / len(df)
null_percents

Patient age quantile                                     0.000000
SARS-Cov-2 exam result                                   0.000000
Patient addmited to regular ward (1=yes, 0=no)           0.000000
Patient addmited to semi-intensive unit (1=yes, 0=no)    0.000000
Patient addmited to intensive care unit (1=yes, 0=no)    0.000000
                                                           ...   
Phosphor                                                 0.996456
ctO2 (arterial blood gas analysis)                       0.995216
InfluenzaAB                                              0.000000
Parainfluenza1234                                        0.000000
Corona                                                   0.000000
Length: 103, dtype: float64

In [9]:
# drop all those column whose null values percentages are more than 80 percent
columns_to_drop = null_percents[null_percents > 0.8].index
df = df.drop(columns=columns_to_drop)


In [10]:
# combine columns of hospitalization
df['Hospitalized'] = df.apply(lambda x: 0 if x['Patient addmited to regular ward (1=yes, 0=no)'] == 0 and
                                                x['Patient addmited to semi-intensive unit (1=yes, 0=no)']==0 and
                                                x['Patient addmited to intensive care unit (1=yes, 0=no)']==0
                              else 1 if x['Patient addmited to regular ward (1=yes, 0=no)'] == 1 
                              else 2 if x['Patient addmited to semi-intensive unit (1=yes, 0=no)']==1
                              else 3 if x['Patient addmited to intensive care unit (1=yes, 0=no)']==1
                              else -1, axis=1)
df.drop(columns='Patient addmited to regular ward (1=yes, 0=no)', inplace=True)
df.drop(columns='Patient addmited to semi-intensive unit (1=yes, 0=no)', inplace=True)
df.drop(columns='Patient addmited to intensive care unit (1=yes, 0=no)',inplace=True)



In [11]:
# count hospitalization by type
df['Hospitalized'].value_counts()

Hospitalized
0    5474
1      79
2      50
3      41
Name: count, dtype: int64

In [12]:
df['InfluenzaAB'].value_counts()

InfluenzaAB
0    5549
2      77
1      18
Name: count, dtype: int64

In [13]:
# Save Sheet to processed.xlsx file
df.to_excel('processed.xlsx')

# Graph on Covid Data

In [ ]:
import matplotlib as plt
import seaborn as sbn

